# Table des Matières
1. [Introduction et Sélection des données](#intro)
2. [Exploration et traitement des données](#traitement)
3. [Modélisation et évaluation](#model)
4. [Comunication des résultats](#comm)
5. [Retour d'expérience](#retour) 

  
 

---

## Introduction et sélection des données <a class="anchor" id="intro"></a>

lien data : https://drive.google.com/file/d/1nUbA6m3SQ9PXpc9i9tD6yADe0fDFYKTc/view?usp=drive_link

Dans le cadre de notre projet de machine Learning, nous allons traiter un sujet d'apprentissage supervisé à partir de données ouvertes. Tout au long de notre projet, nous utiliserons l'IA générative pour nous aider dans notre travail. 

Dans un premier temps, nous commençons par demander à ChatGPT des idées de sujets.

Jeu de données ML généré par l'IA 
- DPE (Analyse énergétique, classification des logements)
- Transports en commun (prédiction affluence, optimisation trajet)
- Pollution (prédiction des pics de pollution, classification zones polluées)
- Reconnaissance d'image
- Traitement du langage naturel
- Donnée de santé
- Donnée financière économique (prédiction de tendance..)

Analyse critique de l'IA
L'IA propose souvent des sujets très classiques, déjà réalisé. Il propose même des projets déjà tout fait sur Kaggle. Il manque d'originalité
Donnée DPE 

Problématique générée avec L'IA :

Peut-on prédire la consommation énergétique d’un logement à partir de ses caractéristiques issues du DPE ?
-> sujet commun en ML

Quels logements sont susceptibles d’être classés comme passoires thermiques (classe F ou G) ?
-> Sujet choisi puisqu'il plus original que les autres 

Peut-on prédire la classe énergétique (A à G) d’un logement à partir des caractéristiques fournies par le DPE ?
-> Sujet déjà réalisé

In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [39]:
df = pd.read_csv('data/data_dpe.csv')

/var/folders/rc/xhmcyg0x1d9b_n3m80ww6sxr0000gn/T/ipykernel_42037/3233007849.py:1: DtypeWarning: Columns (11,48,51,218,219,220,221,222) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/data_dpe.csv')


In [41]:
variables_importantes = [
    # Consommations
    'Conso_5_usages_é_finale', 
    'Conso_5_usages/m²_é_finale', 
    'Conso_chauffage_é_finale', 
    'Conso_ECS_é_finale',
    
    # Isolation
    'Deperditions_baies_vitrées', 
    'Deperditions_enveloppe', 
    'Ubat_W/m²_K', 
    'Qualité_isolation_enveloppe',
    
    # Système de chauffage
    'Type_installation_chauffage', 
    'Type_énergie_principale_chauffage', 
    'Type_émetteur_installation_chauffage_n°1',
    
    # Structure du bâtiment
    'Année_construction', 
    'Période_construction', 
    'Surface_habitable_logement', 
    'Nombre_niveau_logement',
    'Hauteur_sous-plafond',

    
    # Localisation
    'Zone_climatique_', 
    'Classe_altitude', 
    'N°_région_(BAN)', 
    'N°_département_(BAN)',

    # Etiquette 
    'Etiquette_GES',
    'Etiquette_DPE'
]

df_filtered = df[variables_importantes]

## Exploration et traitement des données <a class="anchor" id="traitement"></a>

Critique de l'IA sur notre approche : 
Votre approche est bien structurée et inclut des variables clés dans des domaines pertinents pour prédire si un logement est une passoire thermique. 

Points positifs :
Les variables sélectionnées correspondent bien à des déterminants majeurs de la performance énergétique.
Vous incluez à la fois des variables quantitatives (ex. : consommation énergétique, déperditions) et qualitatives (ex. : type d'installation de chauffage, zone climatique), ce qui permet une modélisation riche.

Critiques et suggestions :

Les bases de données DPE peuvent contenir des valeurs manquantes ou aberrantes (ex. : surfaces non renseignées, consommations incohérentes).
Vérifiez les distributions des variables pour détecter des anomalies.
Plusieurs variables peuvent être fortement corrélées. Une analyse des corrélations peut être utile.
Les passoires thermiques (F et G) pourraient représenter une minorité des données. Cela peut biaiser l’entraînement du modèle.
Les bâtiments anciens sont souvent surreprésentés parmi les passoires thermiques, ce qui peut conduire le modèle à négliger d'autres facteurs. Une analyse exploratoire approfondie est nécessaire.
Les algorithmes robustes pour ce type de problème incluent :
 Forêts aléatoires (Random Forests): pour leur interprétabilité et leur capacité à gérer des variables mixtes.
 Gradient Boosting (XGBoost, LightGBM) : pour leur performance sur des ensembles déséquilibrés.
 Réseaux de neurones : si les données sont enrichies avec des caractéristiques complexes (géographiques, temporelles).